# PRODUCTION phase: About this notebook
- Purpose: Creates 1 AKS webservice, to serve the model as an ONLINE endpoint
    - `AKS ONLINE Webservice:` Fetches the best trained model, Deployes that on an `AKS cluster`, always up and running, ready to be pinged and return results (via REST / Swagger, or Python SDK)

## DETAILS - about this notebook and the 2 pipelines, generated            
- 1) `Initiates ESMLProject` and sets `active model` and `active date folder`:
- 2) `DEPLOY & SERVER: Fetched the BEST MODEL, and deploys on AKS`
- 3) `Smoke testing: Fetches some data and calls the webservice` (smoke testing purpose - see that it works...)
    - Gets test data
    - Calls webservice, which both returns data via REST call, and ESML optionally also saves the returned result to datalake 

# Login / Switch DEV_TEST_PROD environment (1-timer)

In [ ]:
import sys
sys.path.insert(0, "../azure-enterprise-scale-ml/esml/common/")
from azureml.core.authentication import InteractiveLoginAuthentication
from esml import ESMLProject

p = ESMLProject() # Will search in ROOT for your copied SETTINGS folder '../settings/model/active/active_scoring_in_folder.json',
p.dev_test_prod="dev"
auth = InteractiveLoginAuthentication(tenant_id = p.tenant)
#auth = InteractiveLoginAuthentication(force=True, tenant_id = p.tenant)
ws, config_name = p.authenticate_workspace_and_write_config(auth)

# 1) `Initiates ESMLProject` and sets `active model` and `active date folder`:

In [1]:
import sys
sys.path.insert(0, "../azure-enterprise-scale-ml/esml/common/")
from esml import ESMLProject
import pandas as pd

p = ESMLProject() # Will search in ROOT for your copied SETTINGS folder '../settings/model/active/active_scoring_in_folder.json',
p.active_model = 11
p.inference_mode = False
p.ws = p.get_workspace_from_config() #2) Load DEV or TEST or PROD Azure ML Studio workspace
p.verbose_logging = False
#p.describe()

Using lake_settings.json with ESML version 1.4 - Models array support including LABEL


# 2) `DEPLOY & SERVER: Fetched the BEST MODEL, and deploys on AKS`
- Deploy "offline" MODEL from old `run` in environment To →  `DEV`, `TEST` or `PROD` environment
- ESML saves `API_key in Azure keyvault automatically`
- ESML auto-config solves 4 common 'errors/things': `correct compute name` and `valid replicas, valid agents, valid auto scaling`
    - Tip: You can adjust the number of replicas, and different CPU/memory configuration, or using a different compute target.

### Note: AKS_SETTINGS
- Here you can edit AKS settings (performance for DEV, TEST, PROD environments) under PROJECT specific MODEL settings and ONLINE = AKS
    -  Link: [aks_config_dev.json](../settings/project_specific/model/dev_test_prod_override/online/aks_config_dev.json)
- Note: 
    - Q: Why is `docker_bridge_cidr` a PUBLIC IP? Isn't this a PRIVATE AKS cluster?
    - A: Yes, it is PRIVATE. We don’t use the docker bridge for pod communication, but as Docker is configured as part of the Kubernetes setup, this docker bridge it also gets created as well, so in order to avoid that it picks random unknown CIDR that could collide with any of your existent subnets, we give the option to change it and set it a known range. So the indication for docker bridge is to define any CIDR that doesn’t to Azure, and doesn’t collide with any other subnet. 
        - Read more: [learn.microsoft.com](https://learn.microsoft.com/en-us/answers/questions/199786/how-to-update-docker-bridge-cidr-for-aks-to-a-diff.html)

In [ ]:
p.ComputeFactory.aks_config # print settings

In [2]:
import sys
sys.path.insert(0, "../azure-enterprise-scale-ml/")
from esmlrt.interfaces.iESMLController import IESMLController

inference_config, model, best_run = IESMLController.get_best_model_inference_config(p.ws, p.model_folder_name)
# See EXTRA cells in this notebook - if you want to customize the SCORING FILE, in 'inference_config.entry_script'

Searching with Model list LAMBDA FILTER, on experiment_name in Model.tags called: 11_diabetes_model_reg . Meaning ESML checks for both Notebook run (AutoMLRun, Run) and PipelineRuns (AutoMLStep, PipelineRun)
E.g. Even if Pipeline experiment is called '11_diabetes_model_reg_IN_2_GOLD_TRAIN' it will be included, since original model_folder_name in ESML is '11_diabetes_model_reg' as a notebook Run experiment name. Both is included in search
Filter search, minutes: 5.917646221319834
ESML INFO: try: automl_run or notebook_automl
Experiment name: 11_diabetes_model_reg
ws name: msft-weu-DEV-eap-proj02_ai-amls
run_id: c4b9b3f0-a735-4f31-a3e0-e9b94173c2a3


In [ ]:
service,api_uri, kv_aks_api_secret= p.deploy_model_as_private_aks_online_endpoint(model,inference_config,overwrite_endpoint=True)

# 3) `Smoke testing: Fetches some data and calls the webservice`
- Note: In ESML you it also saves the returned result to the datalake as a default behaviour. To turn off this set `save_2_lake_also=False` as below
    - p.call_webservice(ws=p.ws, pandas_X_test=X_test,user_id="10965d9c-40ca-4e47-9723-5a608a32a0e4", firstRowOnly=False,`save_2_lake_also=False`, inference_model_version=None, reload_config=True)

In [ ]:
p.connect_to_lake()
X_test, y_test, tags = p.get_gold_validate_Xy()
print(tags)
caller_id = "10965d9c-40ca-4e47-9723-5a608a32a0e4"

df = p.call_webservice(p.ws, X_test,caller_id) # Saves to datalake also
#df = p.call_webservice(ws=p.ws, pandas_X_test=X_test,user_id=caller_id,firstRowOnly=False,save_2_lake_also=False) # If not saving also to datalake
df.head()

See where the scored data was saved

In [ ]:
to_score_folder, scored_folder, date_folder = p.get_gold_scored_unique_path()
print("Example of where your scored data is saved. Unique folder will be different each time though")
print(scored_folder)
print()
print("Note: Last folder, UUID folder, should represent a 'unique scoring' for a day, but can be injected. Example: if we want a customerGUID instead ")

# END

## EXTRA - use a custom scoring file

In [4]:
import os
os.chdir(os.path.dirname(globals()['_dh'][0]))

scoring_file = "scoring_file_{}.py".format(p.model_folder_name)
script_file_local = "./settings/project_specific/model/dev_test_prod/train/ml/"+scoring_file 
script_file_abs = os.path.abspath(script_file_local)

print("1) Download & EDIT: Local path: to look and edit the file: {}".format(script_file_abs))
best_run.download_file('outputs/scoring_file_v_1_0_0.py', script_file_abs)

print("2) Then set the new scoring script")
#inference_config.entry_script = script_file_abs

print("3) Then Deploy the model")
#service,api_uri, kv_aks_api_secret= p.deploy_model_as_private_aks_online_endpoint(model,inference_config,overwrite_endpoint=True)